In [ ]:
import pandas as pd

class DataProcessor:
    """
    A class to process and merge multiple datasets.

    Attributes:
        kaggle_path (str): Path to the Kaggle dataset file.
        data510_path (str): Path to the '510data' dataset file.
        culture_path (str): Path to the culture dataset file.
    """

    def __init__(self, kaggle_path, data510_path, culture_path):
        """
        Initializes DataProcessor with paths to the datasets.

        Args:
            kaggle_path (str): Path to the Kaggle dataset file.
            data510_path (str): Path to the '510data' dataset file.
            culture_path (str): Path to the culture dataset file.
        """
        self.kaggle_path = kaggle_path
        self.data510_path = data510_path
        self.culture_path = culture_path
    
        
    
    def load_and_clean_kaggle_data(self):
        
        """
        Loads the Kaggle dataset, cleans it by dropping NaNs in 'Life expectancy' 
        and filling NaNs in numeric columns with their mean values.

        Returns:
            DataFrame: Cleaned Kaggle dataset with average values grouped by country.
        """
        kaggle = pd.read_csv(self.kaggle_path)
        kaggle_dropped = kaggle.dropna(subset=['Life expectancy '])
        numeric_columns = kaggle_dropped.select_dtypes(include=['number']).columns

        for var in numeric_columns:
            kaggle_dropped[var].fillna(kaggle_dropped[var].mean(), inplace=True)

        kaggle_dropped_num = kaggle_dropped.drop(columns=['Year', 'Status'])
        kaggle_average = kaggle_dropped_num.groupby('Country').mean().reset_index()
        kaggle_average = kaggle_average.rename(columns={'Country': 'Country Name'})
        kaggle_average['Country Name'] = kaggle_average['Country Name'].str.upper()
        return kaggle_average

    def process_510_data(self):
        """
        Processes the '510data' dataset. This includes renaming columns, converting
        year columns to numeric, and calculating mean values for each country and series.

        Returns:
            DataFrame: Processed '510data' dataset, pivoted with countries and series names.
        """
        df = pd.read_csv(self.data510_path)
        df.columns = [col.split('[')[0].strip() for col in df.columns]
        df = df.drop(columns=['Series Code', 'Country Code'])

        years_columns = [str(year) for year in range(2000, 2016)]
        df[years_columns] = df[years_columns].apply(pd.to_numeric, errors='coerce')
        for col in years_columns:
            df[col] = df.apply(
                lambda row: row[col] if not pd.isna(row[col]) else (
                    row[years_columns].mean() if not row[years_columns].isna().all() else df[col].mean()),
                axis=1
            )

        df['Mean'] = df[years_columns].apply(lambda row: row.mean() if not row.isna().all() else pd.NA, axis=1)
        new_df = df[['Series Name', 'Country Name', 'Mean']]
        agg_df = new_df.groupby(['Country Name', 'Series Name']).mean().reset_index()
        pivoted_df = agg_df.pivot(index='Country Name', columns='Series Name', values='Mean')
        pivoted_df.columns.name = None
        return pivoted_df.reset_index()

    def merge_data(self, kaggle_average, data510_processed):
        """
        Merges the processed Kaggle dataset, '510data' dataset, and culture dataset 
        into a single DataFrame. NaN values in numerical columns are filled with mean values.

        Args:
            kaggle_average (DataFrame): Processed Kaggle dataset with average values.
            data510_processed (DataFrame): Processed '510data' dataset.

        Returns:
            DataFrame: Final merged DataFrame with NaN values filled.
        """
        culture = pd.read_csv(self.culture_path)
        culture.columns.values[0] = 'Country Name'
        culture['Country Name'] = culture['Country Name'].str.upper()

        data510_processed['Country Name'] = data510_processed['Country Name'].str.upper()
        result = data510_processed.merge(culture, how='left', on='Country Name')
        final = result.merge(kaggle_average, how='left', on='Country Name')

        numerical_final = final.select_dtypes(include=['number'])
        mean_values = numerical_final.mean()
        final_clean = final.fillna(mean_values)
        return final_clean

# Usage example
processor = DataProcessor('/Users/shuai/Desktop/Life Expectancy Data.csv', 
                          '/Users/shuai/Desktop/510data.csv', 
                          '/Users/shuai/Desktop/culture_df.csv')

kaggle_average = processor.load_and_clean_kaggle_data()
data510_processed = processor.process_510_data()
final_clean = processor.merge_data(kaggle_average, data510_processed)
final_clean.to_csv('510df_final.csv', index=False)



In [ ]:
# Load the data
file_path = '../data/Cleaned/longevity.csv'
data = pd.read_csv(file_path)

# Convert all values in the Country Name column to lowercase with the first letter of each word in the country name capitalized
data['Country Name'] = data['Country Name'].str.title()

'''
Narrow dataframe to the following columns: 
'''
# narrow dataframe to features of interest
selected_columns = [
    'Country Name',
    'Age at first marriage, female',
    'Age at first marriage, male', 'Capital health expenditure (% of GDP)', 'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)', "Cause of death, by injury (% of total)", 'Cause of death, by non-communicable diseases (% of total)', 'Community health workers (per 1,000 people)', 'Condom use, population ages 15-24, male (% of males ages 15-24)', 'Consumption of iodized salt (% of households)', 'Current health expenditure (% of GDP)', 'Births attended by skilled health staff (% of total)', 'Contraceptive prevalence, any method (% of all women ages 15-49)', 'Current health expenditure per capita (current US$)', 'Demand for family planning satisfied by any methods (% of married women with demand for family planning)', 'Diabetes prevalence (% of population ages 20 to 79)', 'Diarrhea treatment (% of children under 5 receiving oral rehydration and continued feeding)', 'Exclusive breastfeeding (% of children under 6 months)', 'Female headed households (% of households with a female head)', 'Hospital beds (per 1,000 people)', 'Immunization, BCG (% of one-year-old children)', 'Immunization, DPT (% of children ages 12-23 months)', 'Immunization, HepB3 (% of one-year-old children)', 'Immunization, Hib3 (% of children ages 12-23 months)', 'Immunization, Pol3 (% of one-year-old children)', 'Immunization, measles (% of children ages 12-23 months)', 'Immunization, measles second dose (% of children by the nationally recommended age)', 'Domestic private health expenditure per capita (current US$)', 'Domestic general government health expenditure per capita (current US$)', 'Incidence of tuberculosis (per 100,000 people)', 'Literacy rate, adult total (% of people ages 15 and above)', 'Literacy rate, youth total (% of people ages 15-24)', 'Malaria cases reported', 'Mortality caused by road traffic injury (per 100,000 people)', 'Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)', 'Mortality rate attributed to household and ambient air pollution (per 100,000 population)', 'Mortality rate attributed to unintentional poisoning (per 100,000 population)', 'Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)', 
    'Newborns protected against tetanus (%)', 'Number of people who are undernourished', 'Nurses and midwives (per 1,000 people)', 'People practicing open defecation (% of population)', 'People using at least basic drinking water services (% of population)', 'People using at least basic sanitation services (% of population)', 'People using safely managed drinking water services (% of population)', 'People using safely managed sanitation services (% of population)', 'People with basic handwashing facilities including soap and water (% of population)', 'Physicians (per 1,000 people)', 'Poverty headcount ratio at national poverty line (% of population)', 'Prevalence of HIV, total (% of population ages 15-49)', 'Prevalence of current tobacco use (% of adults)', 'Prevalence of hypertension (% of adults ages 30-79)', 'Prevalence of overweight (% of adults)', 'Public spending on education, total (% of GDP)', 'Prevalence of underweight, weight for age (% of children under 5)', 'Prevalence of anemia among women of reproductive age (% of women ages 15-49)', 'Risk of impoverishing expenditure for surgical care (% of people at risk)', 
    'School enrollment, primary (% gross)', 'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)', 'Specialist surgical workforce (per 100,000 population)', 'Suicide mortality rate (per 100,000 population)', 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)', 'Unemployment, total (% of total labor force)', 'Unmet need for contraception (% of married women ages 15-49)', 'Urban poverty headcount ratio at national poverty lines (% of urban population)', 'Rural poverty headcount ratio at national poverty lines (% of rural population)', 'Vitamin A supplementation coverage rate (% of children ages 6-59 months)', 'power_distance', 'individualism', 'motivation', 'uncertainty_avoidance', 'long_term_orientation', 'indulgence', 'Hepatitis B', 'Measles ', ' BMI ', 'Diphtheria ', ' HIV/AIDS', 'GDP', 'Polio', 'Schooling', 'Alcohol', 'Life expectancy at birth, total (years)'
]

data = data[selected_columns]

# Strip all whitespaces from column names
data.columns = data.columns.str.strip()

data.rename(columns={'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)': 'Death due to communicable disease and nutrition conditions'}, inplace =True)

data.rename(columns={'Cause of death, by injury (% of total)': 'Death due to injury'}, inplace =True)

data.rename(columns={'Demand for family planning satisfied by any methods (% of married women with demand for family planning)': 'Demand for family planning'}, inplace =True)

data.rename(columns={'Diabetes prevalence (% of population ages 20 to 79)': 'Diabetes prevalence'}, inplace =True)

data.rename(columns={'Diarrhea treatment (% of children under 5 receiving oral rehydration and continued feeding)': 'Diarrhea treatment'}, inplace =True)

data.rename(columns={'Exclusive breastfeeding (% of children under 6 months)': 'Exclusive breastfeeding'}, inplace =True)

data.rename(columns={'Hospital beds (per 1,000 people)': 'Hospital beds'}, inplace =True)

data.rename(columns={'Immunization, BCG (% of one-year-old children)': 'Immunization, BCG'}, inplace =True)

data.rename(columns={'Immunization, DPT (% of children ages 12-23 months)': 'Immunization, DPT'}, inplace =True)

data.rename(columns={'Immunization, HepB3 (% of one-year-old children)': 'Immunization, HepB3'}, inplace =True)

data.rename(columns={'Immunization, Hib3 (% of children ages 12-23 months)': 'Immunization, Hib3'}, inplace =True)

data.rename(columns={'Immunization, Pol3 (% of one-year-old children)': 'Immunization, Pol3'}, inplace =True)

data.rename(columns={'Immunization, measles (% of children ages 12-23 months)': 'Immunization, measles'}, inplace =True)

data.rename(columns={'Immunization, measles second dose (% of children by the nationally recommended age)': 'Immunization, measles second dose'}, inplace =True)

data.rename(columns={'Domestic private health expenditure per capita (current US$)': 'Domestic private health expenditure'}, inplace =True)

data.rename(columns={'Domestic general government health expenditure per capita (current US$)': 'Domestic general government health expenditure'}, inplace =True)

data.rename(columns={'Incidence of tuberculosis (per 100,000 people)': 'Incidence of tuberculosis'}, inplace =True)

data.rename(columns={'Literacy rate, adult total (% of people ages 15 and above)': 'Literacy rate, adult total'}, inplace =True)

data.rename(columns={'Literacy rate, youth total (% of people ages 15-24)': 'Literacy rate, youth total'}, inplace =True)

data.rename(columns={'Malaria cases reported': 'Malaria cases'}, inplace =True)

data.rename(columns={'Mortality caused by road traffic injury (per 100,000 people)': 'Mortality caused by road traffic injury'}, inplace =True)

data.rename(columns={'Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)': 'Mortality from CVD, cancer, diabetes or CRD'}, inplace =True)

data.rename(columns={'Mortality rate attributed to household and ambient air pollution (per 100,000 population)': 'Mortality rate attributed air pollution'}, inplace =True)

data.rename(columns={'Mortality rate attributed to unintentional poisoning (per 100,000 population)': 'Mortality rate attributed to unintentional poisoning'}, inplace =True)

data.rename(columns={'Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)': 'Mortality rate attributed to unsafe water'}, inplace =True)

data.rename(columns={'Newborns protected against tetanus (%)': 'Newborns protected against tetanus'}, inplace =True)

data.rename(columns={'Number of people who are undernourished': 'Number of undernourished people'}, inplace =True)

data.rename(columns={'Nurses and midwives (per 1,000 people)': 'Nurses and midwives'}, inplace =True)

data.rename(columns={'People practicing open defecation (% of population)': 'People practicing open defecation'}, inplace =True)

data.rename(columns={'People using at least basic drinking water services (% of population)': 'People using at least basic drinking water services'}, inplace =True)

data.rename(columns={'People using at least basic sanitation services (% of population)': 'People using at least basic sanitation services'}, inplace =True)

data.rename(columns={'People using safely managed drinking water services (% of population)': 'People using safely managed drinking water services'}, inplace =True)

data.rename(columns={'People using safely managed sanitation services (% of population)': 'People using safely managed sanitation services'}, inplace =True)

data.rename(columns={'People with basic handwashing facilities including soap and water (% of population)': 'People with basic handwashing facilities'}, inplace =True)

data.rename(columns={'Physicians (per 1,000 people)': 'Physicians'}, inplace =True)

data.rename(columns={'Poverty headcount ratio at national poverty line (% of population)': 'Poverty headcount ratio'}, inplace =True)

data.rename(columns={'Prevalence of HIV, total (% of population ages 15-49)': 'Prevalence of HIV'}, inplace =True)

data.rename(columns={'Prevalence of current tobacco use (% of adults)': 'Prevalence of current tobacco use'}, inplace =True)

data.rename(columns={'Prevalence of hypertension (% of adults ages 30-79)': 'Prevalence of hypertension'}, inplace =True)

data.rename(columns={'Prevalence of overweight (% of adults)': 'Prevalence of overweight'}, inplace =True)

data.rename(columns={'Public spending on education, total (% of GDP)': 'Public spending on education'}, inplace =True)

data.rename(columns={'Prevalence of underweight, weight for age (% of children under 5)': 'Prevalence of underweight children'}, inplace =True)

data.rename(columns={'Prevalence of anemia among women of reproductive age (% of women ages 15-49)': 'Prevalence of anemic women'}, inplace =True)

data.rename(columns={'Risk of impoverishing expenditure for surgical care (% of people at risk)': 'Risk of impoverishing expenditure for surgical care'}, inplace =True)

data.rename(columns={'School enrollment, primary (% gross)': 'School enrollment'}, inplace =True)

data.rename(columns={'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)': 'Total alcohol consumption'}, inplace =True)

data.rename(columns={'Specialist surgical workforce (per 100,000 population)': 'Specialist surgical workforce'}, inplace =True)

data.rename(columns={'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)': 'Teenage mothers'}, inplace =True)

data.rename(columns={'Unemployment, total (% of total labor force)': 'Unemployment'}, inplace =True)

data.rename(columns={'Unmet need for contraception (% of married women ages 15-49)': 'Unmet contraception need for women'}, inplace =True)

data.rename(columns={'Urban poverty headcount ratio at national poverty lines (% of urban population)': 'Urban poverty headcount ratio'}, inplace =True)

data.rename(columns={'Rural poverty headcount ratio at national poverty lines (% of rural population)': 'Rural poverty headcount ratio'}, inplace =True)

data.rename(columns={'Vitamin A supplementation coverage rate (% of children ages 6-59 months)': 'Vitamin A supplementation coverage rate'}, inplace =True)

data.rename(columns={'power_distance': 'Power distribution in society'}, inplace =True)

data.rename(columns={'individualism': 'Individualism'}, inplace =True)

data.rename(columns={'motivation': 'Motivation'}, inplace =True)

data.rename(columns={'uncertainty_avoidance': 'Social uncertainty'}, inplace =True)

data.rename(columns={'long_term_orientation': 'Lack of culture and tradition'}, inplace =True)

data.rename(columns={'indulgence': 'Positivity'}, inplace =True)



# Display the modified DataFrame's first few rows to confirm the changes
data.head()

# save dataframe
data.to_csv('../Data/Cleaned/longevity_reduced.csv', index=False) 